(tutorial-dynmat)=

# Phonon band structure tutorial

In this tutorial you will learn how to use the main workflows of `aiida-quantumespresso-ph`

Let's get started!

In [ ]:
from local_module import load_temp_profile
from aiida.plugins import DataFactory

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
load_temp_profile(
    name="dynmat-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_ph_code=True,
    add_q2r_code=True,
    add_matdyn_code=True,
    add_sssp=True,
)

StructureData = DataFactory("core.structure")
Float = DataFactory("core.float")

## Defining a structure

We need to create or load the structure for which to compute the dynamical matrix. This must be a {py:class}`~aiida.orm.StructureData`.
Here, we define the structure of NaCl salt, which has only 2 atoms in its primitive cell.

Let's define the silicon structure using the ASE module:

In [ ]:
import numpy as np
from ase import Atoms

# Lattice parameter for NaCl (Angstrom)
a = 5.64

# Define primitive cell vectors (in Angstrom)
cell = a * np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0]
])

# Fractional (scaled) positions of basis atoms in the primitive cell
positions = [
    (0.0, 0.0, 0.0),   # Na
    (0.5, 0.5, 0.5)    # Cl
]

# Corresponding atomic symbols
symbols = ['Na', 'Cl']

# Create the primitive NaCl structure
nacl_primitive = Atoms(
    symbols=symbols,
    scaled_positions=positions,
    cell=cell,
    pbc=True
)

structure = StructureData(ase=nacl_primitive)

# Print information about the structure
print(nacl_primitive)
print("Cell vectors (Angstrom):\n", nacl_primitive.get_cell())
print("Atomic positions (Angstrom):\n", nacl_primitive.get_positions())


## Run the `DynamicalMatrixWorkChain` workflow

Let's define the inputs:

* ``pw_code``: the ``Code`` which will run the `pw.x` binary
* ``ph_code``: the ``Code`` which will run the `ph.x` binary
* ``structure``: the StructureData node containing the information regarding the crystal structure

In [ ]:
from aiida.orm import load_code, Dict
from aiida_quantumespresso.common.types import RelaxType, ElectronicType
from aiida_quantumespresso_ph.workflows.dynamical_matrix import DynamicalMatrixWorkChain

pw_code = load_code('pw@localhost')
ph_code = load_code('ph@localhost')

# to make the example run even faster than the 'fast' protocol, we override some parameters
overrides = {
    'relax': {
        'base': {
            'kpoints_distance': 0.6,
        },
    },
    'ph_main': {
        'parallelize_qpoints': False,
        'qpoints_distance': 1.2,
    }
}

builder = DynamicalMatrixWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    ph_code=ph_code,
    structure=structure,
    protocol="fast",
    overrides=overrides,
    **{
        'relax_type': RelaxType.NONE, # do not perform geometry optimization
        'electronic_type': ElectronicType.INSULATOR # NaCl is an insulator
    }
)

And now submit the calculation!

In [ ]:
from aiida.engine import run_get_node

dynmat_results, dynmat_node = run_get_node(builder)

### Inspect the outputs and results

You can now look into the outputs of the workflow.

In [ ]:
print("Is the workflow finished correctly?", dynmat_node.is_finished_ok)

In [ ]:
dynmat_results['ph_output_parameters'].get_dict()

## Run the `PhInterpolateWorkChain` workflow

Let's define the inputs:

* ``q2r_code``: the ``Code`` which will run the `q2r.x` binary
* ``matdyn_code``: the ``Code`` which will run the `matdyn.x` binary
* ``retrieved``: the FolderData node containing the dynamical matrices

In [ ]:
from aiida_quantumespresso_ph.workflows.ph_interpolate import PhInterpolateWorkChain

q2r_code_label = 'q2r@localhost'
matdyn_code_label = 'matdyn@localhost'

### Band path

Define the q-points path using seekpath

In [ ]:
from aiida_quantumespresso.calculations.functions.seekpath_structure_analysis import seekpath_structure_analysis

inputs = {
    'structure': structure,
    'reference_distance': Float(0.02),
}
seekpath_results = seekpath_structure_analysis(**inputs)
band_qpoints = seekpath_results['explicit_kpoints']

### Define the inputs

In [ ]:
inputs = {
    'dynmat_folder': dynmat_node.outputs.ph_retrieved,
    'q2r': {
        'q2r': {
            'code': load_code(q2r_code_label),
            'parameters': Dict({
                'INPUT': {}
            }),
            'metadata': {
                'options': {
                    'resources': {
                        'num_machines': 1,
                        'num_cores_per_mpiproc':1
                    },
                    'max_wallclock_seconds': 10*60,
                },
            },
        },
    },
    'matdyn': {
        'matdyn': {
            'code': load_code(matdyn_code_label),
            'parameters': Dict({
                'INPUT': {
                    'asr': 'simple',
                },
            }),
            'kpoints': band_qpoints,
            'metadata': {
                'options': {
                    'resources': {
                        'num_machines': 1,
                        'num_cores_per_mpiproc': 1
                    },
                    'max_wallclock_seconds': 10*60,
                },
            },
        },
    },
}

### Run the interpolation

In [ ]:
interpolation_results, interpolation_node = run_get_node(PhInterpolateWorkChain, **inputs)

### Plot the phonon band dispersion

In [ ]:
interpolation_results['output_phonon_bands'].show_mpl()

## Inspect other results

In [ ]:
interpolation_node.called[0].outputs.force_constants

In [ ]:
bands = interpolation_results['output_phonon_bands']
bands.get_bands()[0]